# Proyecto 9: Aprendizaje supervisado

## Descripcion del proyecto

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.
Crea un modelo con el máximo valor F1 posible. 

Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 
Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

### Instrucciones del proyecto 

- Descarga y prepara los datos.  Explica el procedimiento.

- Examina el equilibrio de clases. Entrena el modelo sin tener en cuenta el desequilibrio. Describe brevemente tus hallazgos.

- Mejora la calidad del modelo. Asegúrate de utilizar al menos dos enfoques para corregir el desequilibrio de clases. Utiliza conjuntos de entrenamiento y validación para encontrar el mejor modelo y el mejor conjunto de parámetros. Entrena diferentes modelos en los conjuntos de entrenamiento y validación. Encuentra el mejor. Describe brevemente tus hallazgos.

- Realiza la prueba final.

### Descripcion de los datos

Puedes encontrar los datos en el archivo  /datasets/Churn.csv file. Descarga el conjunto de datos.

Características

- RowNumber: índice de cadena de datos
- CustomerId: identificador de cliente único
- Surname: apellido
- CreditScore: valor de crédito
- Geography: país de residencia
- Gender: sexo
- Age: edad
- Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- Balance: saldo de la cuenta
- NumOfProducts: número de productos bancarios utilizados por el cliente
- HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- IsActiveMember: actividad del cliente (1 - sí; 0 - no)
- EstimatedSalary: salario estimado

Objetivo
- Exited: El cliente se ha ido (1 - sí; 0 - no)

## Evaluacion del proyecto

Hemos definido los criterios de evaluación para el proyecto. Lee esto con atención antes de pasar al ejercicio.

Esto es lo que los revisores buscarán cuando evalúen tu proyecto:

- ¿Cómo preparaste los datos para el entrenamiento? ¿Procesaste todos los tipos de características?
- ¿Explicaste los pasos de preprocesamiento lo suficientemente bien?
- ¿Cómo investigaste el equilibrio de clases?
- ¿Estudiaste el modelo sin tener en cuenta el desequilibrio de clases?
- ¿Qué descubriste sobre la investigación del ejercicio?
- ¿Dividiste correctamente los datos en conjuntos?
- ¿Cómo trabajaste con el desequilibrio de clases?
- ¿Utilizaste al menos dos técnicas para corregir el desequilibrio?
- ¿Realizaste correctamente el entrenamiento, la validación y las pruebas finales del modelo?
- ¿Qué tan alto es tu valor F1?
- ¿Examinaste los valores AUC-ROC?
- ¿Mantuviste la estructura del proyecto y el código limpio?
Ya tienes las hojas informativas y los resúmenes de capítulos, tienes todo para continuar con el proyecto.

¡Buena suerte!

## Preparacion de datos

In [36]:
#Importacion de librerias 
# Importar librerias necesarias para el proyecto
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from IPython.display import HTML

In [56]:
# Cargar los datos
data = pd.read_csv("datasets/Churn.csv")

# Por ejemplo, podrías considerar eliminar 'RowNumber', 'CustomerId', y 'Surname' ya que probablemente no sean relevantes para la predicción
data_clean = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Mostrar datos relevantes del dataframe
display(HTML('<hr>'))
display(HTML('<h1> Clientes de Beta-bank '))
display(data_clean.head())
display(data_clean.info())
display(data_clean.describe())
display(HTML('<hr>'))

# Verificacion de datos nulos 
nulos = data_clean.isnull().sum()
display(HTML('<h2> Verificacion de valores nulos'))
display(nulos)

# Verificación de valores duplicados
duplicados = data_clean.duplicated().sum()
display(HTML('<h2> Verificación de valores duplicados </h2>'))
display(f"Total de valores duplicados: {duplicados}")
# Eliminamos valores duplicados
data_clean = data_clean.dropna()

display(HTML('<hr>'))

comentario = """ 
<h2> Comentario sobre la exploracion inicial del archivo </h2>
<p> Al realizar la preparacion de los datos para poder seguir avanzando en el modelo nos dimos que cuanta que este no presenta valores duplicados, pero si nulos, para este caso se decidio descartarlos
dichos datos nulos abarcan 9% de los datos.
<p> Tambien se econtro columnas categoricas las cuales requieren un manejo para poder funcionar en los modelos del machine learning

"""
display(HTML(comentario))


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           9091 non-null   float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


None

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

'Total de valores duplicados: 0'

## Balance de clases

In [38]:
# Examinar balance de clases
class_balance = data_clean['Exited'].value_counts(normalize=True)
print(class_balance)

Exited
0    0.796062
1    0.203938
Name: proportion, dtype: float64


Notamos que hay un claro desequilibrio de clases dentro del tu conjunto de datos. La clase mayoritaria (clientes que no se han dado de baja) constituye una gran mayoría de los casos, mientras que la clase minoritaria (clientes que se han dado de baja) representa una porción mucho menor. Esto significa que si un modelo eligiera la clase más común (prediciendo que todos los clientes se quedarán en el banco), alcanzaría una precisión del 79.6% sin haber aprendido realmente a distinguir entre las características que contribuyen a la deserción de clientes.

## Modelado sin tener en cuenta el desequilibrio de clases

In [39]:
# One-Hot Encoding para variables categóricas
data_encoded = pd.get_dummies(data_clean, drop_first=True)

# División de los datos en características y objetivo
X = data_encoded.drop('Exited', axis=1)
y = data_encoded['Exited']

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345, stratify=y)


In [40]:
# Entrenar un modelo de Regresión Logística
log_reg = LogisticRegression(max_iter=100000, random_state=12345)
log_reg.fit(X_train, y_train)

# Predicción en el conjunto de prueba
y_pred_log_reg = log_reg.predict(X_test)

# Evaluacion 
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg)
roc_auc_log_reg = roc_auc_score(y_test, y_pred_log_reg)

print(f"Accuracy (Regresión Logística): {accuracy_log_reg}")
print(f"F1 Score (Regresión Logística): {f1_log_reg}")
print(f"ROC-AUC Score (Regresión Logística): {roc_auc_log_reg}")

Accuracy (Regresión Logística): 0.8158328752061572
F1 Score (Regresión Logística): 0.3469785575048733
ROC-AUC Score (Regresión Logística): 0.6016449866718292


In [41]:
# Entrenar un modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=12345)
rf.fit(X_train, y_train)

# Predicción y evaluación con el conjunto de prueba
y_pred_rf = rf.predict(X_test)

# Calcular y mostrar el puntaje F1
accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)

print(f"Accuracy (Random Forest): {accuracy_rf}")
print(f"F1 Score (Random Forest): {f1_rf}")
print(f"ROC-AUC Score (Random Forest): {roc_auc_rf}")


Accuracy (Random Forest): 0.8669598680593733
F1 Score (Random Forest): 0.6058631921824105
ROC-AUC Score (Random Forest): 0.7309915340054505


## Interpretacion de los resultados

Los resultados indican que el modelo de Random Forest manejan mejor el problema del desequilibrio entre clases lo cual nos sugiere que este es mas adecuado para predecir la desercion de clientes en el banco.
Ahora con respecto a las evaluaciones:

Accuracy(Precisión): La precision no es una metrica tan relevante para evaluar el rendimiento en conjuntos de datos desequilibrados, sim embargo tener una precision mas alta en un modelo que en otro sugiere que el mas alto es capaz de hacer mejores predicciones generales, para este caso notamos que la precision de Random Forest es de 0.86 que lo deja en posicion alta comparandolo con el modelo de Regresion logistica

F1 Score: Un valor alto en F1 nos indica que el modelo tiene un mejor balance entre precision y sensibilidad lo cual para este contexto es importando porque nos preocupa la clase minoritaria. En este caso el modelo de Random Forest nos presenta una diferencia considerable con respecto al modelo de Regresion Logistica, para Random Forest la puntuacion es de 0.60 y Regresion Logistica 0.34.

ROC-AUC Score: Para esta metrica un valor elevado suguiere que un modelo tiene mejor tasa de verdaderos positivos mientras mantiene baja la tasa de falsos positivos. El modelo de Regresion Logistica obtiene un valor de 0.60 mientras que para Random Forest este valor es mas elevado con 0.73

En conclusion El modelo Random Forest es el que cumple los criterios minimos especificado para la revision del proyecto; Este modelo se puede seguir Mejorando ajustando los hiperparametros.

## Mejorando el modelo Random Forest


In [51]:
# Ajustar Hiperparametros 
# Definir grilla 
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],  
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
}


rf = RandomForestClassifier(random_state=12345, class_weight= 'balanced')

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='f1')

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=12345),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [10, 20, None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='f1', verbose=2)

In [53]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Mejores hiperparámetros:", best_params)

# Evaluación del mejor modelo y prueba final 

y_pred_test = best_model.predict(X_test)

# Calcular métricas de rendimiento
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_test = f1_score(y_test, y_pred_test)
roc_auc_test = roc_auc_score(y_test, y_pred_test)

# Imprimir los resultados de la prueba final
print(f"Accuracy en el conjunto de prueba: {accuracy_test}")
print(f"F1 Score en el conjunto de prueba: {f1_test}")
print(f"ROC-AUC Score en el conjunto de prueba: {roc_auc_test}")

Mejores hiperparámetros: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
Accuracy en el conjunto de prueba: 0.8372732270478285
F1 Score en el conjunto de prueba: 0.6309226932668329
ROC-AUC Score en el conjunto de prueba: 0.7795062620065226


## Resultados 

Tras haber evaluado dos modelos los cuales fueron Random Forest y Logistic Regression se considero Random Forest como el modelo mas optimo para este caso y se siguio trabajando sobre este al punto que se realizaron varios ajustes en los hiperparametros y obtuvimos el mejor modelo y podemos notar que aumenta sus puntuaciones de Precision, F1 y ROC AUC.